#### GRUPO 02: Lubin Ye, Ziteng Huang, Jiahui You

## Introducción al Aprendizaje por Refuerzo en entornos discretos
##### Belén Díaz Agudo 

### Ejemplo 2: Frozen Lake
En el entorno FrozenLake-v0 https://gym.openai.com/envs/FrozenLake-v0/ el agente controla el movimiento de un personaje en un mundo de rejilla. Algunas baldosas son transitables (walkable) y otras hacen que el agente caiga al agua. 
__La dirección de movimiento del agente es incierta y solo depende parcialmente de la dirección elegida (porque puede resbalar en el hielo)__. 
La recompensa se obtiene cuando el agente llega a traves de un camino transitable a una casilla objetivo.

In [1]:
# Creamos el entorno FrozenLake-v1 y realizamos acciones aleatorias para resolver el problema. 
# Se muestra el número de pasos del episodio. 
# Un episodio puede terminar (done) con éxito si ha llegado a la casilla final, o con fallo si ha caido al agua.
# Se ejecutan 20 episodios. Observa si termina con éxito en alguno de ellos. 

# SIN APRENDIZAJE POR REFUERZO


import gym 
import numpy as np

environment_name = "FrozenLake-v1"
env = gym.make(environment_name, is_slippery=True).env
q_table = np.zeros([env.observation_space.n, env.action_space.n])

for i_episode in range(20):
    observation = env.reset()
    for t in range(100):
        env.render()
        print(observation)
        action = env.action_space.sample()
        observation, reward, done, info  = env.step(action)
        
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break
env.close()


SFFF
FHFH
FFFH
HFFG
0
  (Down)
SFFF
FHFH
FFFH
HFFG
1
  (Up)
SFFF
FHFH
FFFH
HFFG
0
  (Left)
SFFF
FHFH
FFFH
HFFG
0
  (Left)
SFFF
FHFH
FFFH
HFFG
0
  (Right)
SFFF
FHFH
FFFH
HFFG
4
  (Down)
SFFF
FHFH
FFFH
HFFG
4
  (Down)
SFFF
FHFH
FFFH
HFFG
4
  (Down)
SFFF
FHFH
FFFH
HFFG
4
Episode finished after 9 timesteps

SFFF
FHFH
FFFH
HFFG
0
  (Down)
SFFF
FHFH
FFFH
HFFG
4
  (Up)
SFFF
FHFH
FFFH
HFFG
0
  (Up)
SFFF
FHFH
FFFH
HFFG
0
  (Right)
SFFF
FHFH
FFFH
HFFG
0
  (Left)
SFFF
FHFH
FFFH
HFFG
4
  (Down)
SFFF
FHFH
FFFH
HFFG
8
Episode finished after 7 timesteps

SFFF
FHFH
FFFH
HFFG
0
  (Up)
SFFF
FHFH
FFFH
HFFG
0
  (Left)
SFFF
FHFH
FFFH
HFFG
0
  (Right)
SFFF
FHFH
FFFH
HFFG
0
  (Down)
SFFF
FHFH
FFFH
HFFG
4
  (Left)
SFFF
FHFH
FFFH
HFFG
8
  (Down)
SFFF
FHFH
FFFH
HFFG
8
  (Left)
SFFF
FHFH
FFFH
HFFG
8
Episode finished after 8 timesteps

SFFF
FHFH
FFFH
HFFG
0
  (Right)
SFFF
FHFH
FFFH
HFFG
0
  (Right)
SFFF
FHFH
FFFH
HFFG
1
Episode finished after 3 timesteps

SFFF
FHFH
FFFH
HFFG
0
  (Down)
SFFF
FHFH
FFFH
HFFG
1
  (D

In [2]:
# Observamos el tamaño del espacio de estados
print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

Action Space Discrete(4)
State Space Discrete(16)


In [48]:
#APRENDIZAJE POR REFUERZO

import random
from IPython.display import clear_output

def trainAgent(q_table,alpha,gamma,epsilon,iterations=100000):
    """Training the agent"""
    # For plotting metrics
    all_epochs = []
    all_penalties = []

    for i in range(1, iterations+1):
        state = env.reset()

        epochs, penalties, reward, = 0, 0, 0
        done = False

        while not done:
            if random.uniform(0, 1) < epsilon:
                action = env.action_space.sample() # Explore action space
            else:
                action = np.argmax(q_table[state]) # Exploit learned values

            next_state, reward, done, info = env.step(action) 

            # Redefinimos la función de recompensa
            if reward == 0 and not done:
                reward = -1
            
            if reward == 0 and done:
                reward = -10
            
            if reward == 1 and done:
                reward = 100
            
            old_value = q_table[state, action]
            next_max = np.max(q_table[next_state])

            new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
            q_table[state, action] = new_value

            if reward == -10:
                penalties += 1

            state = next_state
            epochs += 1

        if i % 100 == 0:
            clear_output(wait=True)
            print(f"Episode: {i}")

    print("Training finished.\n")

In [41]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames,time=.2):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        #print(frame['frame'].getvalue())
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(time)

In [64]:
import gym

def solveFrozenLake(q_table,state,showF=False,time=.2):
    """Solves the Frozen Lake Problem"""
    
    env = gym.make("FrozenLake-v1", is_slippery=True).env
    env.s=state  # el estado de la imagen
    env.render()
    epochs = 0
    reward = 0

    frames = [] # for animation
    done = False

    while not done:
        action = np.argmax(q_table[int(state)])
        state, reward, done, info = env.step(action)
        
        # Put each rendered frame into dict for animation
        frames.append({
            'frame': env.render(mode='ansi'),
            'state': state,
            'action': action,
            'reward': reward
            }
        )

        epochs += 1
        
        if done or epochs >= 100:
            done = True
            print("Timesteps taken: {}".format(epochs))

    
    if (showF):
        print_frames(frames,time)
        
    env.close()

    return reward

In [67]:
%%time
q_table = np.zeros([env.observation_space.n, env.action_space.n])
trainAgent(q_table,0.1,0.9,1,10000)

Episode: 10000
Training finished.

CPU times: total: 250 ms
Wall time: 2.23 s


In [68]:
# Dado que la recompensa es 1 a la hora de resolver el problema, podemos usarlo para contar las veces 
# que llega al estado objetivo

rewards = 0
n = 10

for _ in range(n):
    reward = solveFrozenLake(q_table,0,True)
    rewards += reward

print("Times found goal: {}".format(rewards))

  (Down)
SFFF
FHFH
FFFH
HFFG

Timestep: 31
State: 15
Action: 1
Reward: 1.0
Times found goal: 9.0


### Ejercicio: configura el aprendizaje por refuerzo para resolver el problema de Frozen Lake.
#### Utiliza distintos valores de configuración de los parámetros y observa qué configuración se comporta mejor. ¿qué métrica has utilizado para determinar si una configuración es buena? 
#### Observa cuál es la función de recompensa que se define por defecto (consulta en la documentación de gym).  Mejórala reescribiendo el valor de reward y observa cómo afecta la mejora de la función de recompensa en el proceso de aprendizaje.  Explica cómo has medido esta mejora.


#### Puedes entregar tus resultados de forma opcional: 

- Métrica utilizada: 
La métrica que se ha empleado para saber si la configuración es buena es que el número de éxitos de llegar al estado objetivo tras ejecutar la búsqueda n veces sea mayor que 3n/4.

- Función de recompensa mejorada: 
Crear una recompensa de valor mayor al llegar al estado final (100) y penalizar (-10) si se cae al agujero 

- Mejor configuración de parámetros 
alfa=0.1, gamma=0.9, epsilon=1, iteraciones=10000

- Resumen de resultados (los mejores obtenidos) con la métrica anterior: 
En el caso de nuestro experimiento hemos decidido fijar n=10, donde nuestro mejor resultado ha sido de 9 éxitos en 10 resoluciones del problema, que sería un acierto del 90%.
